In [1]:
# bibliotecas
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import numpy as np

from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models

from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_auc_score

### Carregamento e Pré-Processamento do Dataset

In [2]:
from utils.data_utils_keras import load_and_preprocess_data_tf, ModelType

batch_size = 32
image_size = (224, 224)
train_ds, test_ds = load_and_preprocess_data_tf(resize_to=image_size,
                                                batch_size=batch_size,
                                                model_type=ModelType.RESNET50)

Found 100000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.


### Divisão Holdout

In [3]:
total_batches = tf.data.experimental.cardinality(train_ds).numpy()
total_samples = total_batches * batch_size

In [4]:
# Define proporção de treino (70%) e validação (30%) usando o número de batches
train_batches = int(total_batches * 0.7)
val_batches = total_batches - train_batches

In [5]:
# Embaralha os batches do dataset
train_ds_shuffled = train_ds.shuffle(buffer_size=1000, seed=42, reshuffle_each_iteration=False)

In [6]:
# Divisão trenio e validação
train_ds = train_ds_shuffled.take(train_batches)
val_ds = train_ds_shuffled.skip(train_batches)

train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [7]:
print(f"Batches de treino: {train_batches}")
print(f"Batches de validação: {val_batches}")
print(f"Aproximadamente {train_batches * batch_size} amostras de treino")
print(f"Aproximadamente {val_batches * batch_size} amostras de validação")

Batches de treino: 2187
Batches de validação: 938
Aproximadamente 69984 amostras de treino
Aproximadamente 30016 amostras de validação


## Treino

### Modelo

**ResNet50**

O ResNet50 é uma arquitetura de rede neural convolucional profunda de ponta, desenvolvida pela Microsoft Research em 2015. É uma variante da popular arquitetura ResNet e compreende 50 camadas que permitem o aprendizado de arquiteturas muito mais profundas do que era possível anteriormente, sem o problema de gradientes que desaparecem. 

A arquitetura do ResNet50 é dividida em quatro partes principais: as camadas convolucionais, o bloco de identidade, o bloco convolucional e as camadas totalmente conectadas. As camadas convolucionais são responsáveis ​​por extrair características da imagem de entrada, o bloco de identidade e o bloco convolucional processam e transformam essas características, e as camadas totalmente conectadas fazem a classificação final. O ResNet50 foi treinado no grande conjunto de dados ImageNet, alcançando uma taxa de erro equivalente ao desempenho humano, tornando-o um modelo poderoso para diversas tarefas de classificação de imagens, como detecção de objetos, reconhecimento facial e análise de imagens médicas. Além disso, também tem sido usado como um extrator de características para outras tarefas, como detecção de objetos e segmentação semântica.

Fonte: https://medium.com/@nitishkundu1993/exploring-resnet50-an-in-depth-look-at-the-model-architecture-and-code-implementation-d8d8fa67e46f

In [ ]:
device = "GPU" if len(tf.config.list_physical_devices("GPU")) > 0 else "CPU"
print(f"Usando: {device}")

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(*image_size, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    batch_size=batch_size,
    verbose=1
)

In [ ]:
y_pred_prob = model.predict(test_ds)
y_pred = (y_pred_prob > 0.5).astype(int).ravel()

# Extrair labels verdadeiros do dataset de teste
y_test = []
for _, labels in test_ds:
    y_test.extend(labels.numpy())
y_test = np.array(y_test)

# Flatten y_pred_prob para garantir formato correto
y_pred_prob = y_pred_prob.ravel()

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão")
plt.show()

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=["Real", "Fake"]))

roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC (ROC): {roc_auc:.4f}")

plt.figure(figsize=(8, 4))

# Acurácia
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title("Curva de Aprendizado - Acurácia")
plt.xlabel("Época")
plt.ylabel("Acurácia")
plt.legend()
plt.grid()

# Perda
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title("Curva de Aprendizado - Perda")
plt.xlabel("Época")
plt.ylabel("Perda (Loss)")
plt.legend()
plt.grid()

plt.show()